# Data_analyze_prj

**Задание:**   
1) разбить столбец `info` на другие столбцы и таким образом избавится от него  
2) переформатировать столбец `time`, разбить его на столбец `time` и `date`  
3) упаковать проект с помощью cookiecutter

### Data Sources
- [link1](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/data/synscan.csv): Репозиторий

### Changes
12-01-2020: Начало работы  
12-01-2020: Окончание работы

In [18]:
import pandas as pd
import numpy as np
import re
import datetime

In [19]:
df = pd.read_csv("https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/data/synscan.csv")

In [20]:
df["tcp_retransmission"] = df.Info.str.find("[TCP Retransmission]")  
df["tcp_retransmission"] = np.where(df["tcp_retransmission"] == 0, True, False)
df["tcp_retransmission"]

0       False
1       False
2       False
3       False
4       False
        ...  
2006     True
2007     True
2008     True
2009     True
2010     True
Name: tcp_retransmission, Length: 2011, dtype: bool

In [3]:
df.columns = [x.lower() for x in df.columns]

In [4]:
df["info"] = df["info"].str.replace(r"[TCP Retransmission]", "", regex=False) 
df["info"]

0       36050  >  443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
1       36050  >  143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
2       36050  >  3306 [SYN] Seq=0 Win=2048 Len=0 MSS=...
3       36050  >  199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
4       36050  >  111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460
                              ...                        
2006     80  >  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 ...
2007     22  >  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 ...
2008     53  >  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 ...
2009     80  >  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 ...
2010     22  >  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 ...
Name: info, Length: 2011, dtype: object

In [5]:
df['info'] = df['info'].str.replace(r" > ", "", regex=False)
df['info']

0          36050  443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
1          36050  143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
2         36050  3306 [SYN] Seq=0 Win=2048 Len=0 MSS=1460
3          36050  199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460
4          36050  111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460
                              ...                        
2006     80  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 Len...
2007     22  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 Len...
2008     53  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 Len...
2009     80  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 Len...
2010     22  36050 [SYN, ACK] Seq=0 Ack=1 Win=5840 Len...
Name: info, Length: 2011, dtype: object

In [6]:
df["info"] = df["info"].str.strip()  

In [7]:
import datetime
date_time = (pd.Period(datetime.datetime.now(), 'M') - 1).strftime("%Y-%m-%d %H:%M")   
format_date_time = (pd.to_datetime(date_time) + pd.to_timedelta(df['time'].to_numpy(), 'D')) 
df['date'] = format_date_time.strftime('%Y-%m-%d')    
df['time'] = format_date_time.strftime('%H:%M:%S') 

In [8]:
df = df.reindex(columns=['no.', 'date', 'time', 'source', 'destination', 'protocol', 'length', 'info', 'tcp_retransmission'])
df.head()

,no.,date,time,source,destination,protocol,length,info,tcp_retransmission
0,1,2020-11-30,00:00:00,172.16.0.8,64.13.134.52,TCP,58,36050 443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
1,2,2020-11-30,00:02:12,172.16.0.8,64.13.134.52,TCP,58,36050 143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
2,3,2020-11-30,00:02:17,172.16.0.8,64.13.134.52,TCP,58,36050 3306 [SYN] Seq=0 Win=2048 Len=0 MSS=1460,False
3,4,2020-11-30,00:02:22,172.16.0.8,64.13.134.52,TCP,58,36050 199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
4,5,2020-11-30,00:02:27,172.16.0.8,64.13.134.52,TCP,58,36050 111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460,False


In [9]:
sawl = [[sawl.split('=')[0] for sawl in row.split() if '=' in sawl] for row in df['info'].to_list()]
flatten = lambda l: [item for sublist in l for item in sublist]
all_s = set(flatten(sawl))
all_s

{'Ack', 'Len', 'MSS', 'Seq', 'Win'}

In [10]:
import re
def fun_sawl(row, sawl):
    return re.search(f'{sawl}='+r'\d*', row).group(0).split('=')[-1] if sawl in row else False
for sawl in ('Seq', 'Win', 'Len', 'MSS', 'Ack'):
    df[sawl.lower()] = df['info'].map(lambda x, y = sawl: fun_sawl(x, y))
df.head()

,no.,date,time,source,destination,protocol,length,info,tcp_retransmission,seq,win,len,mss,ack
0,1,2020-11-30,00:00:00,172.16.0.8,64.13.134.52,TCP,58,36050 443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False
1,2,2020-11-30,00:02:12,172.16.0.8,64.13.134.52,TCP,58,36050 143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False
2,3,2020-11-30,00:02:17,172.16.0.8,64.13.134.52,TCP,58,36050 3306 [SYN] Seq=0 Win=2048 Len=0 MSS=1460,False,0,2048,0,1460,False
3,4,2020-11-30,00:02:22,172.16.0.8,64.13.134.52,TCP,58,36050 199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False
4,5,2020-11-30,00:02:27,172.16.0.8,64.13.134.52,TCP,58,36050 111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460,False,0,1024,0,1460,False


In [11]:
def fun(row):
    return re.search(r"\[([A-Za-z0-9_, A-Za-z0-9_]+)\]", row).group(0)

In [12]:
df['flag'] = df['info'].map(fun)
df.head()

,no.,date,time,source,destination,protocol,length,info,tcp_retransmission,seq,win,len,mss,ack,flag
0,1,2020-11-30,00:00:00,172.16.0.8,64.13.134.52,TCP,58,36050 443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN]
1,2,2020-11-30,00:02:12,172.16.0.8,64.13.134.52,TCP,58,36050 143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN]
2,3,2020-11-30,00:02:17,172.16.0.8,64.13.134.52,TCP,58,36050 3306 [SYN] Seq=0 Win=2048 Len=0 MSS=1460,False,0,2048,0,1460,False,[SYN]
3,4,2020-11-30,00:02:22,172.16.0.8,64.13.134.52,TCP,58,36050 199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN]
4,5,2020-11-30,00:02:27,172.16.0.8,64.13.134.52,TCP,58,36050 111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460,False,0,1024,0,1460,False,[SYN]


In [13]:
def get_port(row, port):
    return row.split()[:2][port] 

In [14]:
df['source_port'] = df['info'].map(lambda r, p = 0: get_port(r, p))
df['destination_port'] = df['info'].map(lambda r, p = 1: get_port(r, p))
df.head()

,no.,date,time,source,destination,protocol,length,info,tcp_retransmission,seq,win,len,mss,ack,flag,source_port,destination_port
0,1,2020-11-30,00:00:00,172.16.0.8,64.13.134.52,TCP,58,36050 443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN],36050,443
1,2,2020-11-30,00:02:12,172.16.0.8,64.13.134.52,TCP,58,36050 143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN],36050,143
2,3,2020-11-30,00:02:17,172.16.0.8,64.13.134.52,TCP,58,36050 3306 [SYN] Seq=0 Win=2048 Len=0 MSS=1460,False,0,2048,0,1460,False,[SYN],36050,3306
3,4,2020-11-30,00:02:22,172.16.0.8,64.13.134.52,TCP,58,36050 199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False,0,3072,0,1460,False,[SYN],36050,199
4,5,2020-11-30,00:02:27,172.16.0.8,64.13.134.52,TCP,58,36050 111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460,False,0,1024,0,1460,False,[SYN],36050,111


In [15]:
df = df.drop('info', 1)  
df.head()

,no.,date,time,source,destination,protocol,length,tcp_retransmission,seq,win,len,mss,ack,flag,source_port,destination_port
0,1,2020-11-30,00:00:00,172.16.0.8,64.13.134.52,TCP,58,False,0,3072,0,1460,False,[SYN],36050,443
1,2,2020-11-30,00:02:12,172.16.0.8,64.13.134.52,TCP,58,False,0,3072,0,1460,False,[SYN],36050,143
2,3,2020-11-30,00:02:17,172.16.0.8,64.13.134.52,TCP,58,False,0,2048,0,1460,False,[SYN],36050,3306
3,4,2020-11-30,00:02:22,172.16.0.8,64.13.134.52,TCP,58,False,0,3072,0,1460,False,[SYN],36050,199
4,5,2020-11-30,00:02:27,172.16.0.8,64.13.134.52,TCP,58,False,0,1024,0,1460,False,[SYN],36050,111


In [16]:
df.to_csv(r'отчет.csv')